<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 The MediaPipe Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with MediaPipe Model Maker

In this colab notebook, you'll learn how to use [MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker) to train a custom object detection model to detect dogs.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset to use with the [MediaPipe Object Detector task](https://developers.google.com/mediapipe/solutions/vision/object_detector).


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/google/mediapipe/tree/master/mediapipe/model_maker) and the pycocotools library you'll use for evaluation.

In [2]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

Import the required packages.

In [3]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from google.colab import files

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


### Prepare the dataset

This dataset contains about 130 images of 3 different dogs: a Golden Retriever, a Labrador Retriever mix, and a brindle-colored Formosan Mountain Dog. This is an example image from the dataset.

![aci.jpg](https://storage.googleapis.com/tflite-task-tfjs/aci.jpeg)

We start with downloading the dataset.

In [4]:
!wget https://storage.googleapis.com/mediapipe-assets/dogs2.zip --no-check-certificate
!unzip dogs2.zip
train_dataset_path = "dogs/train"
validation_dataset_path = "dogs/validate"

--2023-03-31 17:15:51--  https://storage.googleapis.com/mediapipe-assets/dogs2.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.135.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58926065 (56M) [application/zip]
Saving to: ‘dogs2.zip’

dogs2.zip           100%[===================>]  56.20M  65.1MB/s    in 0.9s    

2023-03-31 17:15:52 (65.1 MB/s) - ‘dogs2.zip’ saved [58926065/58926065]

Archive:  dogs2.zip
   creating: dogs copy/
   creating: dogs copy/validate/
  inflating: dogs copy/.DS_Store     
  inflating: __MACOSX/dogs copy/._.DS_Store  
   creating: dogs copy/train/
  inflating: dogs copy/validate/.DS_Store  
  inflating: __MACOSX/dogs copy/validate/._.DS_Store  
   creating: dogs copy/validate/images/
   creating: dogs copy/validate/Annotations/
  inflating: dogs copy/train/.DS_Store  
  inflating: __MACOSX/dogs c

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [5]:
train_data = object_detector.Dataset.from_pascal_voc_folder(
    'dogs copy/train',
    cache_dir="/tmp/od_data/train",
)

val_data = object_detector.Dataset.from_pascal_voc_folder(
    'dogs copy/validate',
    cache_dir="/tmp/od_data/validatation")

### Step 2: Train the TensorFlow model with the training data.

* Set `batch_size = 8` here so you will see that it takes 13 steps to go through the 139 images in the training dataset.
* Set `learning_rate = 0.3` here so the model will adjust its weights at a rate of 0.3.
* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.


In [6]:
hparams = object_detector.HParams(batch_size=8, learning_rate=0.3, epochs=50, learning_rate_boundaries=[20],
                                  learning_rate_decay_multipliers=[1.0], export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=object_detector.SupportedModels.MOBILENET_V2,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

/usr/local/lib/python3.9/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 24)  2257984   
                             , '3': (None, 32, 32, 32            
                             ),                                  
                              '4': (None, 16, 16, 96)            
                             , '5': (None, 8, 8, 320)            
                             , '6': (None, 8, 8, 1280            
                             )}                                  
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128),  149056    
                              '4': (None, 16, 16, 128            
                             ),                                  
                              '3': (None, 32, 32, 128            
                             ),                   

/usr/local/lib/python3.9/dist-packages/keras/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


13/13 [==============================] - 70s 1s/step - total_loss: 6.0486 - cls_loss: 5.7378 - box_loss: 0.0051 - model_loss: 5.9931 - val_total_loss: 11.8829 - val_cls_loss: 5.1048 - val_box_loss: 0.1344 - val_model_loss: 11.8264
Epoch 2/50
13/13 [==============================] - 5s 411ms/step - total_loss: 2.3447 - cls_loss: 1.9451 - box_loss: 0.0068 - model_loss: 2.2872 - val_total_loss: 15.4439 - val_cls_loss: 9.4128 - val_box_loss: 0.1195 - val_model_loss: 15.3855
Epoch 3/50
13/13 [==============================] - 4s 341ms/step - total_loss: 1.8362 - cls_loss: 1.4971 - box_loss: 0.0056 - model_loss: 1.7773 - val_total_loss: 9.0063 - val_cls_loss: 6.5915 - val_box_loss: 0.0471 - val_model_loss: 8.9471
Epoch 4/50
13/13 [==============================] - 6s 484ms/step - total_loss: 1.3456 - cls_loss: 1.0733 - box_loss: 0.0043 - model_loss: 1.2861 - val_total_loss: 6.4653 - val_cls_loss: 4.8766 - val_box_loss: 0.0306 - val_model_loss: 6.4057
Epoch 5/50
13/13 [=======================

### Step 3. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 27 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [7]:
loss, coco_metrics = model.evaluate(val_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

7/7 [==============================] - 3s 89ms/step - total_loss: 0.9751 - cls_loss: 0.7497 - box_loss: 0.0033 - model_loss: 0.9155
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.15s).
Accumulating evaluation results...
DONE (t=0.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.423
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.638
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.511
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.441
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.472
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.607
 Average Recall     (AR) @[ Io

### Step 4: Export as a TensorFlow Lite model.

After creating the model, convert and export it to a Tensorflow Lite model format, and then download it for later use in an on-device application with MediaPipe Tasks.

In [13]:
model.export_model('dogs.tflite')
!ls exported_model
files.download('exported_model/dogs.tflite')

Exporting a floating point model


/usr/local/lib/python3.9/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.9/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


dogs.tflite			float_ckpt.index  model.tflite
float_ckpt.data-00000-of-00001	metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>